# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [40]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [41]:
from itertools import groupby
import numpy as np

In [42]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        # Trainning set
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.train = self.full_dictionary[:]
        
        # Base characters
        self.char_set = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',\
                         's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
        
        # Unigram 
        self.unigram_len, self.unigram_tol = self.unigram_probs()
        
        # ngram sets with grams > 1
        self.bigram, self.trigram, self.fourgram, self.fivegram = self.build_ngrams()
        
        # Guessed char
        self.guessed_char = []
        
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []      
    
    def calc_tol_prob(self, word_set):#input is a word set regardless of word length. return the probability of each letter A-Z in this dictionary
        '''
        calculate the probability of each letter given a word_set
        Input: word_set is a list of words
        Output: an 26*1 np.array of probs, corresponding from a to z
        '''
        big_string = "".join(word_set) # Merge into a big string
        probs_l = []
        tol = len(big_string)
        if tol == 0:
            return np.zeros(26)
        for i in range(26):
            probs_l.append(big_string.count(self.char_set[i]) / tol)
        probs_l = np.array(probs_l)

        return probs_l

    def unigram_probs(self):#row 1: words with length = 1*a-z probs，row 2：words with len = 2*a-z probs，...28 rows
        '''
        Input: A list of words
        Return: 2 array, len_probs and tol_probs
        len_probs: A n*26 list, each row represents a letter, from a to z. Each column represents a probability, 
                  corresponding to a length orderd from 1 to the length of the longest string in the training set.
        tol_probs: A 26, list, each entry is a probability corresponding to a to z
        '''
        temp_train = self.train[:]
        
        # tol_probs (ignore length of words)
        tol_probs = self.calc_tol_prob(temp_train)

        # frequency with different length of words
        # sorts the temp_train list by the length of each word, with the shortest words first and the longest words last
        temp_train.sort(key=len)
        groupByLen = [list(set(items)) for length, items in groupby(temp_train, key=len)]
        
        len_probs = []
        for i in range(len(groupByLen)):
            len_probs.append(self.calc_tol_prob(groupByLen[i]))

        return len_probs, tol_probs    
    
    def unigram_predict(self, word):#word example: _ _ _ _ _ _ _ _ _ _ _ _ _ _
        '''
        when we know nothing, the first letter prediction
        input: a string. A blankword with all '_', each represents an unknown letter
        output: a char. A guess
        '''
        
        # Case when the length of word is in our prob set
        if(len(word) <= len(self.len_probs)):
            probs = len_probs[len(word)-1]
            ind = probs.index(max(probs))#长度一样的单词里选出现频率最多的letter为第一个predict
            pred = letter_set[ind]
        if pred != '':
            return pred
        else:
            ind = tol_probs.index(max(tol_probs))#猜一个长度为29的单词，选整个字典里出现概率最高的letter
            pred = letter_set[ind]
            return pred
        
    def build_ngrams(self):
        bigram = []
        trigram = []
        fourgram = []
        fivegram = []
        for word in self.train:
            #bigram
            for i in range(len(word)-1):
                bigram.append(word[i:i+2])
            #trigram
            for i in range(len(word)-2):
                trigram.append(word[i:i+3])
            #fourgram
            for i in range(len(word)-3):
                fourgram.append(word[i:i+4])
            #fivegram
            for i in range(len(word)-4):
                fivegram.append(word[i:i+5])
        return bigram, trigram, fourgram, fivegram
    
    def bigram_scores(self, word):
        '''
        input: a string with length 2, such as '_c','c_',return a score for this bigram.
        '''
        
        # '_c' case
        if (word[0] == '_' and word[1] != '_'):
            # a list of bigram in such form
            matches = [bi for bi in self.bigram if bi[1] == word[1]] 
            chars = [string[0] for string in matches]
            joined_matches = "".join(chars)
            tol_probs = self.calc_tol_prob(joined_matches)
            tol_scores = tol_probs
        # 'c_' case
        elif(word[1] == '_' and word[0] != '_'):
            matches = [bi for bi in self.bigram if bi[0] == word[0]]
            chars = [string[1] for string in matches]
            joined_matches = "".join(chars)
            tol_probs = self.calc_tol_prob(joined_matches)
            tol_scores = tol_probs
        else:
            return np.zeros(26)
    
        return tol_scores
    
    def trigram_scores(self, word):
        '''
        input: a string with length 3, such as '_cc','c_c','cc_', '__c','c__'
        note that the last two cases are considered with one letter and two blanks.
        return a score for this bigram.
        '''
        # remove all the space
        word = word.replace(" ", "")
        
        flag0 = True if word[0] != '_' else False
        flag1 = True if word[1] != '_' else False
        flag2 = True if word[2] != '_' else False
        # '_cc' case
        if (flag0 == False and (flag1*flag2)):
            matches = [tri for tri in self.trigram if (tri[1] == word[1] and tri[2] == word[2])] # add self. to bigram
            chars = [string[0] for string in matches]
            joined_matches = "".join(chars)
            tol_scores = self.calc_tol_prob(joined_matches)

        # 'c_c' case
        elif (flag1 == False and (flag0 * flag2)):
            matches = [tri for tri in self.trigram if (tri[0] == word[0] and tri[2] == word[2])] # add self. to bigram
            chars = [string[1] for string in matches]
            joined_matches = "".join(chars)
            # Trigram probs
            tol_scores = self.calc_tol_prob(joined_matches)

        # 'cc_' case
        elif(flag2 == False and (flag0 * flag1)):
            matches = [tri for tri in self.trigram if (tri[0] == word[0] and tri[1] == word[1])] # add self. to bigram
            chars = [string[2] for string in matches]
            joined_matches = "".join(chars)
            # Trigram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        else:
            return np.zeros(26)

        return tol_scores
    
    
    def fourgram_scores(self,word):
        # remove all the space
        word = word.replace(" ", "")
        
        flag0 = False if word[0] == '_' else True
        flag1 = False if word[1] == '_' else True
        flag2 = False if word[2] == '_' else True
        flag3 = False if word[3] == '_' else True
        # '_ccc' case
        if (flag0 == False and (flag1*flag2*flag3)):
            matches = [f for f in self.fourgram if (f[1] == word[1] and f[2] == word[2] and f[3] == word[3])] # add self. to bigram
            chars = [string[0] for string in matches]
            joined_matches = "".join(chars)
            # Fourgram probs
            tol_scores = self.calc_tol_prob(joined_matches)

        # 'c_cc' case
        elif(flag1 == False and (flag0 * flag2 * flag3)):
            matches = [f for f in self.fourgram if (f[0] == word[0] and f[2] == word[2] and f[3] == word[3])] 
            chars = [string[1] for string in matches]
            joined_matches = "".join(chars)
            # Fourgram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        # 'cc_c' case
        elif(flag2 == False and (flag0 * flag1 * flag3)):
            matches = [f for f in self.fourgram if (f[0] == word[0] and f[1] == word[1] and f[3] == word[3])] 
            chars = [string[2] for string in matches]
            joined_matches = "".join(chars)
            # Fourgram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        # 'ccc_' case
        elif(flag3 == False and (flag0 * flag1 * flag2)):
            matches = [f for f in self.fourgram if (f[0] == word[0] and f[1] == word[1] and f[2] == word[2])] 
            chars = [string[3] for string in matches]
            joined_matches = "".join(chars)
            # Fourgram probs
            tol_scores = self.calc_tol_prob(joined_matches)

        else:
            return np.zeros(26)

        return tol_scores
    
    
    def fivegram_scores(self, word):
        # remove all the space
        word = word.replace(" ", "")
        
        flag0 = True if word[0] != '_' else False
        flag1 = True if word[1] != '_' else False
        flag2 = True if word[2] != '_' else False
        flag3 = True if word[3] != '_' else False
        flag4 = True if word[4] != '_' else False

        # '_cccc' case
        if (flag0 == False and flag1*flag2*flag3*flag4):
            matches = [f for f in self.fivegram if (f[1] == word[1] and f[2] == word[2] \
                                               and f[3] == word[3] and f[4] == word[4])] # add self. to bigram
            chars = [string[0] for string in matches]
            joined_matches = "".join(chars)
            # Fivegram probs
            tol_scores = self.calc_tol_prob(joined_matches)

        # 'c_ccc' case
        elif (flag1 == False and flag0*flag2*flag3*flag4):
            matches = [f for f in self.fivegram if (f[0] == word[0] and f[2] == word[2] \
                                               and f[3] == word[3] and f[4] == word[4])] # add self. to bigram
            chars = [string[1] for string in matches]
            joined_matches = "".join(chars)
            # Fivegram probs
            tol_scores = self.calc_tol_prob(joined_matches)

        # 'cc_cc' case
        elif (flag2 == False and flag0*flag1*flag3*flag4):
            matches = [f for f in self.fivegram if (f[0] == word[0] and f[1] == word[1] \
                                               and f[3] == word[3] and f[4] == word[4])] # add self. to bigram
            chars = [string[2] for string in matches]
            joined_matches = "".join(chars)
            # Fivegram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        # 'ccc_c' case
        elif (flag3 == False and flag0*flag1*flag2*flag4):
            matches = [f for f in self.fivegram if (f[0] == word[0] and f[1] == word[1] \
                                               and f[2] == word[2] and f[4] == word[4])] # add self. to bigram
            chars = [string[3] for string in matches]
            joined_matches = "".join(chars)
            # Fivegram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        # 'cccc_' case
        elif(flag4 == False and flag0*flag1*flag2*flag3):
            matches = [f for f in self.fivegram if (f[0] == word[0] and f[1] == word[1] \
                                               and f[2] == word[2] and f[3] == word[3])] # add self. to bigram
            chars = [string[4] for string in matches]
            joined_matches = "".join(chars)
            # Fivegram probs
            tol_scores = self.calc_tol_prob(joined_matches)
            
        else:
            return np.zeros(26) 
        return tol_scores

    def guess(self, word):
        '''
        Input: A string. The word to gusse.Example: "_ p p _ e "
        Output: a char. The guess
        '''
        word = word.replace(" ", "")
        #print(word)
        
        #      a           b    ...      z
        #    prob_a1   prob_a2   ...  prob_zn
        probs = np.zeros(26)
        
        # Unigram
        unigram_probs = None
        if len(word) <= len(self.unigram_len):
            unigram_probs = self.unigram_len[len(word)-1]
        else:
            unigram_probs = self.unigram_tol
        probs = probs + unigram_probs*0.1
        
        # Bigram
        for i in range(len(word)-1):
            probs += self.bigram_scores(word[i:i+2])*0.3
        # Trigram
        for i in range(len(word)-2):
            probs += self.trigram_scores(word[i:i+3])*0.9
        # Fourgram
        for i in range(len(word)-3):
            probs += self.fourgram_scores(word[i:i+4])*3.2
        # Fivegram
        for i in range(len(word)-4):
            probs += self.fivegram_scores(word[i:i+5])*5.4
        
        for ind in self.guessed_char:
            probs[ind] = probs[ind]*0
            
        pred = self.char_set[list(probs).index(max(probs))]
        self.guessed_char.append(self.char_set.index(pred))
        
        return pred
        

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    HANGMAN_URL + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [11]:
api = HangmanAPI(access_token="8264c783bafbe6d90d65c46eab35df", timeout=2000)

In [12]:
import time

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [33]:
api.my_status()

[722, 0, 0, 245]

In [34]:
api = HangmanAPI(access_token="8264c783bafbe6d90d65c46eab35df", timeout=2000)
for i in range(20):
    api.guessed_char = []
    start_time = time.time()
    api.start_game(practice=1, verbose=True)
    # Get my game stats: (# of tries, # of wins)
    [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status() 
    practice_success_rate = (total_practice_successes-245) / (total_practice_runs-722)
    print('run %d practice games out of an allotted 100. practice success rate so far = %.3f' \
          % (total_practice_runs, practice_success_rate))
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")

Successfully start a new game! Game ID: d3c113cb869d. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e r _ _ _ _ '}
Guessing letter: t
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ t e r _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ t e r _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n t e r _ _ _ n '}
Guessing letter: i
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i n t e r _ _ i n '}
Guessing letter: o
Sever response: {'game_id': 'd3c113cb869d', 'status': 'ongoing', 'tries_

Guessing letter: s
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ _ _ _ s e _ _ _ _ _ s '}
Guessing letter: i
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 4, 'word': 'e _ _ _ _ s e _ _ _ _ _ s '}
Guessing letter: n
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 3, 'word': 'e _ _ _ _ s e _ _ _ _ _ s '}
Guessing letter: l
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 2, 'word': 'e _ _ _ _ s e _ _ _ _ _ s '}
Guessing letter: a
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 2, 'word': 'e _ _ _ _ s e _ a _ _ _ s '}
Guessing letter: m
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 2, 'word': 'e m _ _ _ s e m a _ _ _ s '}
Guessing letter: t
Sever response: {'game_id': 'f60b54dd51ea', 'status': 'ongoing', 'tries_remains': 2, 'word': 'e m _ _ _ s e m a t _ _ s '}
Guessi

Sever response: {'game_id': '78c5e33f8e68', 'status': 'failed', 'tries_remains': 0, 'word': '_ _ s '}
Failed game: 78c5e33f8e68. Because of: # of tries exceeded!
run 732 practice games out of an allotted 100. practice success rate so far = 0.700
Elapsed time: 3.106847047805786 seconds
Successfully start a new game! Game ID: fe778c8032bf. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: a
Sever response: {'game_id': 'fe778c8032bf', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': 'fe778c8032bf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'fe778c8032bf', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ '}
Guessing letter: o
Sever response: {'game_id': 'fe778c8032bf', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o _ o _ '}
Guessing letter: l
Sever response: {'game_id': 'fe778c8032bf', 'status': 'ongoing', 'tries_remains': 3, 'word': '

Successfully start a new game! Game ID: f355e988956d. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: a
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ '}
Guessing letter: r
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e r r _ '}
Guessing letter: i
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e r r i '}
Guessing letter: t
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e r r i '}
Guessing letter: b
Sever response: {'game_id': 'f355e988956d', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e r r i '}
Guessing letter: f
Sever response: {'game_id': 'f355e988956d', 'status': 'success', 'tries_remains': 3, 'word': 'f e r r i '}
Successfully finished

Guessing letter: y
Sever response: {'game_id': '8dcfc9bd92d7', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o _ y c h r o n i c o n '}
Guessing letter: l
Sever response: {'game_id': '8dcfc9bd92d7', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o l y c h r o n i c o n '}
Guessing letter: p
Sever response: {'game_id': '8dcfc9bd92d7', 'status': 'success', 'tries_remains': 3, 'word': 'p o l y c h r o n i c o n '}
Successfully finished game: 8dcfc9bd92d7
run 742 practice games out of an allotted 100. practice success rate so far = 0.600
Elapsed time: 15.963079929351807 seconds


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [44]:
api = HangmanAPI(access_token="8264c783bafbe6d90d65c46eab35df", timeout=2000)

In [46]:
api.my_status()

[742, 3, 0, 257]

In [48]:

for i in range(1000):
    api.guessed_char = []
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status() 
    recorded_success_rate = (total_recorded_successes) / (total_recorded_runs)
    print('accuracy = %.3f' % recorded_success_rate)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
accuracy = 0.000
Playing  1  th game
accuracy = 0.167
Playing  2  th game
accuracy = 0.143
Playing  3  th game
accuracy = 0.250
Playing  4  th game
accuracy = 0.333
Playing  5  th game
accuracy = 0.400
Playing  6  th game
accuracy = 0.455
Playing  7  th game
accuracy = 0.500
Playing  8  th game
accuracy = 0.462
Playing  9  th game
accuracy = 0.500
Playing  10  th game
accuracy = 0.533
Playing  11  th game
accuracy = 0.500
Playing  12  th game
accuracy = 0.471
Playing  13  th game
accuracy = 0.444
Playing  14  th game
accuracy = 0.421
Playing  15  th game
accuracy = 0.400
Playing  16  th game
accuracy = 0.381
Playing  17  th game
accuracy = 0.364
Playing  18  th game
accuracy = 0.348
Playing  19  th game
accuracy = 0.333
Playing  20  th game
accuracy = 0.320
Playing  21  th game
accuracy = 0.346
Playing  22  th game
accuracy = 0.370
Playing  23  th game
accuracy = 0.393
Playing  24  th game
accuracy = 0.379
Playing  25  th game
accuracy = 0.400
Playing  26  th game
a

Playing  213  th game
accuracy = 0.509
Playing  214  th game
accuracy = 0.511
Playing  215  th game
accuracy = 0.509
Playing  216  th game
accuracy = 0.511
Playing  217  th game
accuracy = 0.514
Playing  218  th game
accuracy = 0.516
Playing  219  th game
accuracy = 0.518
Playing  220  th game
accuracy = 0.520
Playing  221  th game
accuracy = 0.522
Playing  222  th game
accuracy = 0.524
Playing  223  th game
accuracy = 0.522
Playing  224  th game
accuracy = 0.524
Playing  225  th game
accuracy = 0.526
Playing  226  th game
accuracy = 0.524
Playing  227  th game
accuracy = 0.522
Playing  228  th game
accuracy = 0.519
Playing  229  th game
accuracy = 0.521
Playing  230  th game
accuracy = 0.523
Playing  231  th game
accuracy = 0.525
Playing  232  th game
accuracy = 0.527
Playing  233  th game
accuracy = 0.529
Playing  234  th game
accuracy = 0.531
Playing  235  th game
accuracy = 0.533
Playing  236  th game
accuracy = 0.531
Playing  237  th game
accuracy = 0.533
Playing  238  th game
acc

accuracy = 0.558
Playing  424  th game
accuracy = 0.557
Playing  425  th game
accuracy = 0.558
Playing  426  th game
accuracy = 0.557
Playing  427  th game
accuracy = 0.556
Playing  428  th game
accuracy = 0.554
Playing  429  th game
accuracy = 0.555
Playing  430  th game
accuracy = 0.554
Playing  431  th game
accuracy = 0.555
Playing  432  th game
accuracy = 0.554
Playing  433  th game
accuracy = 0.555
Playing  434  th game
accuracy = 0.554
Playing  435  th game
accuracy = 0.552
Playing  436  th game
accuracy = 0.551
Playing  437  th game
accuracy = 0.552
Playing  438  th game
accuracy = 0.553
Playing  439  th game
accuracy = 0.554
Playing  440  th game
accuracy = 0.555
Playing  441  th game
accuracy = 0.554
Playing  442  th game
accuracy = 0.553
Playing  443  th game
accuracy = 0.551
Playing  444  th game
accuracy = 0.552
Playing  445  th game
accuracy = 0.551
Playing  446  th game
accuracy = 0.552
Playing  447  th game
accuracy = 0.551
Playing  448  th game
accuracy = 0.552
Playing 

Playing  634  th game
accuracy = 0.546
Playing  635  th game
accuracy = 0.547
Playing  636  th game
accuracy = 0.546
Playing  637  th game
accuracy = 0.545
Playing  638  th game
accuracy = 0.546
Playing  639  th game
accuracy = 0.547
Playing  640  th game
accuracy = 0.546
Playing  641  th game
accuracy = 0.546
Playing  642  th game
accuracy = 0.547
Playing  643  th game
accuracy = 0.546
Playing  644  th game
accuracy = 0.545
Playing  645  th game
accuracy = 0.546
Playing  646  th game
accuracy = 0.545
Playing  647  th game
accuracy = 0.544
Playing  648  th game
accuracy = 0.545
Playing  649  th game
accuracy = 0.544
Playing  650  th game
accuracy = 0.544
Playing  651  th game
accuracy = 0.544
Playing  652  th game
accuracy = 0.543
Playing  653  th game
accuracy = 0.544
Playing  654  th game
accuracy = 0.543
Playing  655  th game
accuracy = 0.544
Playing  656  th game
accuracy = 0.545
Playing  657  th game
accuracy = 0.544
Playing  658  th game
accuracy = 0.544
Playing  659  th game
acc

accuracy = 0.552
Playing  845  th game
accuracy = 0.552
Playing  846  th game
accuracy = 0.552
Playing  847  th game
accuracy = 0.552
Playing  848  th game
accuracy = 0.551
Playing  849  th game
accuracy = 0.552
Playing  850  th game
accuracy = 0.552
Playing  851  th game
accuracy = 0.553
Playing  852  th game
accuracy = 0.552
Playing  853  th game
accuracy = 0.552
Playing  854  th game
accuracy = 0.552
Playing  855  th game
accuracy = 0.551
Playing  856  th game
accuracy = 0.552
Playing  857  th game
accuracy = 0.551
Playing  858  th game
accuracy = 0.550
Playing  859  th game
accuracy = 0.551
Playing  860  th game
accuracy = 0.550
Playing  861  th game
accuracy = 0.550
Playing  862  th game
accuracy = 0.549
Playing  863  th game
accuracy = 0.550
Playing  864  th game
accuracy = 0.550
Playing  865  th game
accuracy = 0.551
Playing  866  th game
accuracy = 0.550
Playing  867  th game
accuracy = 0.549
Playing  868  th game
accuracy = 0.549
Playing  869  th game
accuracy = 0.549
Playing 

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [38]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.000


In [39]:
api.my_status()

[742, 3, 0, 257]